In [29]:
import xml.etree.ElementTree
import os, os.path, shutil
os.sep = '/'
from gedcom import Gedcom, GedcomReader, GedcomWriter

def copy_tree_image(img_path):
    shutil.copy(src=img_path, dst='static/tree_img'+os.path.splitext(img_path)[-1])
    shutil.copy(src=os.path.splitext(img_path)[0]+'.xml', dst='data/tree_img.xml')

def set_default_documents_for_persons(gedcom, treexml):
    '''для каждого документа персоны находим "основной" и проставляем ему тег default (DFLT)'''
    # составляем словарь uid персоны -> title основного документа
    default_documents = dict()
    for person in treexml.iter('r'):
        person_uid = person.attrib['id']
        for document in person.iter('doc'):
            if document.get('default') == "T":
                title_element = document.find('title')
                if title_element is not None:
                    title = title_element.text
                    default_documents[person_uid] = title
                    break
                else:
                    print('Документ установленный как основной для ['+person.find('fullname').text +'] не имеет заголовка')
    # по созданному списку выставляем аттрибут DFLT для основных документов персоны
    for person in gedcom.persons:
        person_uid = person.get('_UID')
        if person_uid not in default_documents:
            continue
        has_default = False
        for document in person.documents:
            title = document.get('TITL')
            if default_documents[person_uid] == title:
                document['DFLT'] = 'T'
                has_default = True
                break
        if not has_default and len(person.documents) > 0:
            print('Person '+person_uid+' has not default document with title '+ default_documents[person_uid])

def set_divorce_events_to_families(gedcom, treexml):
    '''Устанавниваем событие развод для семьи (оно есть в treexml, но отсутствует в gedcom)'''
    # по gedcom составляем список (uid-супруга1, uid-супруга2) -> family
    person_id_to_uid = dict()
    for person in gedcom.persons:
        person_id_to_uid[person.id] = person.get('_UID', 'unknown')
    
    families_by_uids = dict()
    for family in gedcom.families:
        husb_uid = person_id_to_uid.get(family.get('HUSB'), '')
        wife_uid = person_id_to_uid.get(family.get('WIFE'), '')
        
        families_by_uids[(husb_uid, wife_uid)] = family
        families_by_uids[(wife_uid, husb_uid)] = family

        
    # по xml находим у людей события divorce
    for person in treexml.iter('r'):
        person_uid = person.attrib['id']
        for spouse in person.iter('spouse'):
            spouse_uid = spouse.get('id')
            family = families_by_uids.get((person_uid, spouse_uid)) or families_by_uids.get((spouse_uid, person_uid))
            if family is None:
                print('Family ', (person_uid, spouse_uid), ' is not found')
                continue
            divorce = spouse.find('divorce')
            if divorce is not None:
                divorce_event = Gedcom.Event('DIV')
                date = divorce.find('date')
                if date is not None:
                    divorce_event['DATE'] = date.text
                comment = divorce.find('comment')
                if comment is not None:
                    divorce_event['NOTE'] = comment
                family.events.append(divorce_event)

def set_documents_to_marriage(gedcom):
    '''документы указанные как документы семьи на самом деле относятся к событию свадьба'''
    for family in gedcom.families:
        marrs = list(filter(lambda event: event.event_type=='MARR', family.events))
        if len(marrs) > 0:
            marr = marrs[0]
            marr.documents = family.documents
            family.documents = []

import random, string
def random_string(length, letters=None):
    letters = string.ascii_lowercase + string.digits
    return ''.join(random.choice(letters) for i in range(length))
            
def pack_documents(gedcom, folder):
    if os.path.exists(folder):
        shutil.rmtree(folder)
    os.makedirs(folder)
    relative_pathes = {}
    for person in gedcom.persons:
        person_uid = person.get('_UID')
        person_folder = os.path.join(folder, person_uid)
        all_documents = person.documents + [doc for event in person.events for doc in event.documents]
        for document in all_documents:
            document_path = document.get('FILE')
            if document_path is None or not os.path.exists(document_path):
                document['FILE'] = ""
                continue
            if document_path not in relative_pathes:
                ext = os.path.splitext(document_path)[-1]
                new_document_name = random_string(10)+ext
                if not os.path.exists(person_folder):
                    os.makedirs(person_folder)
                new_document_path = os.path.join(person_folder, new_document_name)   
                shutil.copy(src=document_path, dst=new_document_path)
                rel_document_path = os.path.join(person_uid, new_document_name)
                relative_pathes[document_path] = rel_document_path
            
            document['FILE'] = relative_pathes[document_path]


In [30]:
# читаем gedcom
gedcom = GedcomReader().read_gedcom(r'D:/Документы/My Family/site/tree.ged')
treexml = xml.etree.ElementTree.parse('D:/Документы/My Family/site/tree.xml').getroot()

copy_tree_image(r'D:/Документы/My Family/site/tree_img.png')
set_default_documents_for_persons(gedcom, treexml)
set_documents_to_marriage(gedcom)
set_divorce_events_to_families(gedcom, treexml)
pack_documents(gedcom, 'static/tr_tree.files')

GedcomWriter().write_gedcom(gedcom, 'data/tr_tree.ged')

In [109]:
import pandas
events = pandas.read_csv('data/events.csv', sep=';', skiprows=1)
persons = pandas.read_csv('data/persons.csv', sep=';', skiprows=1)
documents = pandas.read_csv('data/documents.csv', sep=';', skiprows=1)

gedcom = GedcomReader().read_gedcom('data/exmp.ged')
print(len(gedcom.persons))
for family in gedcom.families:
    print(family)

0


In [39]:
from app_utils import first_or_default

gedcom = GedcomReader().read_gedcom('data/exmp.ged')
person = gedcom.persons[0]
print([(e.event_type, e) for e in person.events])
death_event = first_or_default(person.events, lambda e: e.event_type == "DEAT")
print(death_event)
if death_event is not None:
    print('death')

[('BIRT', {'NOTE': 'Старший брат'}), ('DEAT', {}), ('RESI', {'PLAC': 'Россия, Свердловская обл., г.Среднеуральск'})]
{}
death
